In [ ]:
# Title:  Process NYPD Arrest Dataset CSV for OLAP use
# Author: Andrey Norin
# Date:   12/7/2022

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
# import original csv
#df = pd.read_csv('NYPD_Arrests_Data__Historic_.csv')

In [ ]:
df = pd.read_parquet('NYPD_Arrests_Data__Historic.parquet')

In [ ]:
# drop unneeded columns
df = df.drop(columns=['JURISDICTION_CODE','PD_DESC','PD_CD','KY_CD','LAW_CODE','PERP_RACE',
'X_COORD_CD','Y_COORD_CD','Latitude','Longitude','Lon_Lat'])

In [ ]:
# translate codes into meaningful values
df['LAW_CAT_CD'] = df['LAW_CAT_CD'].replace(['M','F','V'], ['Misdemeanor','Felony','Violation'])
df['ARREST_BORO'] = df['ARREST_BORO'].replace(['Q','M','B','K','S'],['Queens','Manhattan','Bronx','Brooklyn','Staten Island'])

In [ ]:
# generate IDs for date
df['OFFENSE_ID'] = "OFID" + df.ARREST_KEY.map(str)
df['DATE_ID'] = "DTID" + df.ARREST_KEY.map(str)

In [ ]:
# convert column names all column names to lowercase
df.columns= df.columns.str.lower()

In [ ]:
# convert the 'Date' column to datetime format
df['arrest_date']= pd.to_datetime(df['arrest_date'])

In [ ]:
# explode arrest_date into multiple columns
df['arrest_day']      = df['arrest_date'].map(lambda x: x.day)
df['arrest_month']    = df['arrest_date'].map(lambda x: x.strftime("%B"))
df['arrest_year']     = df['arrest_date'].map(lambda x: x.year)
df['arrest_day_name'] = df['arrest_date'].map(lambda x: x.strftime("%A"))

In [ ]:
# rearrange column order into final configuration
# use print(df.columns.to_list()) to obtain list of columns

cols = ['arrest_key', 'offense_id', 'ofns_desc', 'law_cat_cd', 'arrest_boro', 'arrest_precinct',
'age_group', 'perp_sex','date_id','arrest_date', 'arrest_day', 'arrest_month', 'arrest_year',
'arrest_day_name']
df = df.reindex(columns=cols)
df

In [ ]:
# export df
df.to_parquet('nypd_arrest_dataset_processed.parquet')
df.to_csv('nypd_arrest_dataset_processed.csv')